In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


import ipykernel
import pandas as pd

#pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 800)
pd.set_option('display.max_colwidth', 500)
pd.set_option('expand_frame_repr', True)


In [2]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F


device  = torch.device("cuda:1" if torch.cuda.is_available() else "cuda:0")
device

device(type='cuda', index=1)

In [3]:
import json
import random
import matplotlib.pyplot as plot
import numpy as np
import os
import jsonlines
import ast
import time, os, fnmatch, shutil
from collections import defaultdict
import matplotlib
import requests

import math
import random
import copy

from collections import namedtuple, deque

import numpy as np


HISTORY_FILE = None

summary_dict = {"losses":0,"wins":0,"draws":0}

cycle = 0

actions = [[], []]
rewards = [[], []]
history = {"cycle": cycle, "actions": actions, "rewards": rewards, "value": 0}

score_board = {}
#step_action_dict = defaultdict()
#step_action_dict['random_moves'] = defaultdict()
#step_action_dict['policy_moves'] = defaultdict()






In [4]:
%%writefile numeric_names.py

numeric_names = {'w_R0': 1, 'w_K0': 2, 'w_B0': 3, 'w__K': 4, 'w__Q': 5, 'w_B1': 6, 'w_K1': 7, 'w_R1': 8, 'w_P0': 9, 'w_P1':
 10, 'w_P2': 11, 'w_P3': 12, 'w_P4': 13, 'w_P5': 14, 'w_P6': 15, 'w_P7': 16, 'b_P0': -16, 'b_P1': -15, 'b_P2': -14, 'b_P3':
 -13, 'b_P4': -12, 'b_P5': -11, 'b_P6': -10, 'b_P7': -9, 'b_R0': -8, 'b_K0': -7, 'b_B0': -6, 'b__K': -5, 'b__Q': -4, 'b_B1'
: -3, 'b_K1': -2, 'b_R1': -1}

Overwriting numeric_names.py


In [5]:
from collections import defaultdict
from games import games
from numeric_names import numeric_names

Pieces_detail = defaultdict()
actions_array = []
sparse_action_dict = defaultdict()
bin_action_dict = defaultdict()
actions = []

for piece in numeric_names.keys():
    action_sparse = []
    if piece[0] == 'w':
        idx = 0
    else:
        idx = 1
        
    if piece[1:3] == '_R':
        player_id = 'Rook'
    elif piece[1:3] == '_K':
        player_id = 'Knight'
    elif piece[1:3] == '_B':
        player_id = 'Bishop'
    elif piece[1:3] == '_P':
        player_id = 'Pawn'
    elif piece[1:4] == '__K':
        player_id = 'King'
    elif piece[:3] == '__Q':
        player_id = 'Queen'
        
    actions = games['chessy']['players'][player_id]['moves']
    
    actions = actions[idx]
    
    #action_diff = 4 - len(actions)
    
    #for i in range(action_diff):
    #    actions.append((0,0))
    
    piece_num_id = numeric_names[piece]
    
    piece_bin_id = '{0:06b}'.format(piece_num_id + 17)
    
    actions_ = []
    
    for action in actions:
        action_value_  = (8*action[0] + action[1] + 17)
        action_value   = '{0:06b}'.format(action_value_)
        actions_.append(action_value)
        
        piece_move_bin = (str(piece_bin_id)+str(action_value))
        
        actions_array.append(piece_move_bin)
        
        sparse_action  = str(piece_num_id)+","+str(action[0])+ "," +str(action[1])
        
        action_sparse.append(sparse_action)
        
        sparse_action_dict[sparse_action] = piece_move_bin
    
    #print(piece_num_id, "\t",piece_bin_id, "\t", action_sparse,  "\t", actions, "\t", actions_)
    
    Pieces_detail[piece_num_id] = {
        'piece_num_id':piece_num_id,
        'piece_bin_id':piece_bin_id,
        'action_sparse':action_sparse,
        'actions_verbose': actions,
        'actions_bin': actions_
       }
    
action_ids = np.arange(0,len(sparse_action_dict.keys()),1)
#print(action_ids)


action_id_dict = {y:x for x,y in zip(action_ids,sparse_action_dict.keys())}
print(action_id_dict) 

id_action_dict = {x:y for x,y in zip(action_ids,sparse_action_dict.keys())}
print(id_action_dict)

{'1,1,0': 0, '2,1,-2': 1, '2,1,2': 2, '2,2,-1': 3, '2,2,1': 4, '3,1,-1': 5, '3,1,1': 6, '4,1,-1': 7, '4,1,0': 8, '4,1,1': 9, '5,1,-1': 10, '5,1,0': 11, '5,1,1': 12, '6,1,-1': 13, '6,1,1': 14, '7,1,-2': 15, '7,1,2': 16, '7,2,-1': 17, '7,2,1': 18, '8,1,0': 19, '9,1,0': 20, '9,1,-1': 21, '9,1,1': 22, '10,1,0': 23, '10,1,-1': 24, '10,1,1': 25, '11,1,0': 26, '11,1,-1': 27, '11,1,1': 28, '12,1,0': 29, '12,1,-1': 30, '12,1,1': 31, '13,1,0': 32, '13,1,-1': 33, '13,1,1': 34, '14,1,0': 35, '14,1,-1': 36, '14,1,1': 37, '15,1,0': 38, '15,1,-1': 39, '15,1,1': 40, '16,1,0': 41, '16,1,-1': 42, '16,1,1': 43, '-16,-1,0': 44, '-16,-1,-1': 45, '-16,-1,1': 46, '-15,-1,0': 47, '-15,-1,-1': 48, '-15,-1,1': 49, '-14,-1,0': 50, '-14,-1,-1': 51, '-14,-1,1': 52, '-13,-1,0': 53, '-13,-1,-1': 54, '-13,-1,1': 55, '-12,-1,0': 56, '-12,-1,-1': 57, '-12,-1,1': 58, '-11,-1,0': 59, '-11,-1,-1': 60, '-11,-1,1': 61, '-10,-1,0': 62, '-10,-1,-1': 63, '-10,-1,1': 64, '-9,-1,0': 65, '-9,-1,-1': 66, '-9,-1,1': 67, '-8,-1,0': 

In [6]:
wins_draws_losses = [0,0,0]

In [7]:
%%writefile args.py


import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from numeric_names import numeric_names

args = { 
    "update_every":100,
    "window_size":100,
    "BUFFER_SIZE":int(1e6),
    "BATCH_SIZE":1024,  
    "GAMMA":0.99,
    "TAU":2e-3,
    "LR_ACTOR":1e-3,
    "LR_CRITIC":1.1e-3,
    "WEIGHT_DECAY":0.0001,
    "UPDATE_EVERY":5,
    "EXPLORE_NOISE":0.05,
    "FC1_UNITS":10240,
    "FC2_UNITS":4096,
    "FC3_UNITS":256,
    "seed":0,
    "state_size":384,
    "action_size":88,
    "action_size_binary":12,
    "num_agents":2,
    "device":torch.device("cuda:1" if torch.cuda.is_available() else "cuda:0"),
    'mcritic_path':'/home/ubuntu/chessy/checkpoint_mCritic_twoSided.pth',
    'agent_p0_path':'/home/ubuntu/chessy/checkpoint_p0_twoSided.pth',
    'agent_p1_path':'/home/ubuntu/chessy/checkpoint_p1_twoSided.pth',
    'action_id_dict': {'1,1,0': 0, '2,1,-2': 1, '2,1,2': 2, '2,2,-1': 3, '2,2,1': 4, '3,1,-1': 5, '3,1,1': 6, '4,1,-1': 7, '4,1,0': 8, '4,1,1': 9, '5,1,-1': 10, '5,1,0': 11, '5,1,1': 12, '6,1,-1': 13, '6,1,1': 14, '7,1,-2': 15, '7,1,2': 16, '7,2,-1': 17, '7,2,1': 18, '8,1,0': 19, '9,1,0': 20, '9,1,-1': 21, '9,1,1': 22, '10,1,0': 23, '10,1,-1': 24, '10,1,1': 25, '11,1,0': 26, '11,1,-1': 27, '11,1,1': 28, '12,1,0': 29, '12,1,-1': 30, '12,1,1': 31, '13,1,0': 32, '13,1,-1': 33, '13,1,1': 34, '14,1,0': 35, '14,1,-1': 36, '14,1,1': 37, '15,1,0': 38, '15,1,-1': 39, '15,1,1': 40, '16,1,0': 41, '16,1,-1': 42, '16,1,1': 43, '-16,-1,0': 44, '-16,-1,-1': 45, '-16,-1,1': 46, '-15,-1,0': 47, '-15,-1,-1': 48, '-15,-1,1': 49, '-14,-1,0': 50, '-14,-1,-1': 51, '-14,-1,1': 52, '-13,-1,0': 53, '-13,-1,-1': 54, '-13,-1,1': 55, '-12,-1,0': 56, '-12,-1,-1': 57, '-12,-1,1': 58, '-11,-1,0': 59, '-11,-1,-1': 60, '-11,-1,1': 61, '-10,-1,0': 62, '-10,-1,-1': 63, '-10,-1,1': 64, '-9,-1,0': 65, '-9,-1,-1': 66, '-9,-1,1': 67, '-8,-1,0': 68, '-7,-1,-2': 69, '-7,-1,2': 70, '-7,-2,-1': 71, '-7,-2,1': 72, '-6,-1,-1': 73, '-6,-1,1': 74, '-5,-1,-1': 75, '-5,-1,0': 76, '-5,-1,1': 77, '-4,-1,-1': 78, '-4,-1,0': 79, '-4,-1,1': 80, '-3,-1,-1': 81, '-3,-1,1': 82, '-2,-1,-2': 83, '-2,-1,2': 84, '-2,-2,-1': 85, '-2,-2,1': 86, '-1,-1,0': 87},
    'id_action_dict': {0: '1,1,0', 1: '2,1,-2', 2: '2,1,2', 3: '2,2,-1', 4: '2,2,1', 5: '3,1,-1', 6: '3,1,1', 7: '4,1,-1', 8: '4,1,0', 9: '4,1,1', 10: '5,1,-1', 11: '5,1,0', 12: '5,1,1', 13: '6,1,-1', 14: '6,1,1', 15: '7,1,-2', 16: '7,1,2', 17: '7,2,-1', 18: '7,2,1', 19: '8,1,0', 20: '9,1,0', 21: '9,1,-1', 22: '9,1,1', 23: '10,1,0', 24: '10,1,-1', 25: '10,1,1', 26: '11,1,0', 27: '11,1,-1', 28: '11,1,1', 29: '12,1,0', 30: '12,1,-1', 31: '12,1,1', 32: '13,1,0', 33: '13,1,-1', 34: '13,1,1', 35: '14,1,0', 36: '14,1,-1', 37: '14,1,1', 38: '15,1,0', 39: '15,1,-1', 40: '15,1,1', 41: '16,1,0', 42: '16,1,-1', 43: '16,1,1', 44: '-16,-1,0', 45: '-16,-1,-1', 46: '-16,-1,1', 47: '-15,-1,0', 48: '-15,-1,-1', 49: '-15,-1,1', 50: '-14,-1,0', 51: '-14,-1,-1', 52: '-14,-1,1', 53: '-13,-1,0', 54: '-13,-1,-1', 55: '-13,-1,1', 56: '-12,-1,0', 57: '-12,-1,-1', 58: '-12,-1,1', 59: '-11,-1,0', 60: '-11,-1,-1', 61: '-11,-1,1', 62: '-10,-1,0', 63: '-10,-1,-1', 64: '-10,-1,1', 65: '-9,-1,0', 66: '-9,-1,-1', 67: '-9,-1,1', 68: '-8,-1,0', 69: '-7,-1,-2', 70: '-7,-1,2', 71: '-7,-2,-1', 72: '-7,-2,1', 73: '-6,-1,-1', 74: '-6,-1,1', 75: '-5,-1,-1', 76: '-5,-1,0', 77: '-5,-1,1', 78: '-4,-1,-1', 79: '-4,-1,0', 80: '-4,-1,1', 81: '-3,-1,-1', 82: '-3,-1,1', 83: '-2,-1,-2', 84: '-2,-1,2', 85: '-2,-2,-1', 86: '-2,-2,1', 87: '-1,-1,0'},
    'numeric_names' : numeric_names,
    'initial_state'  : "010010010011010100010101010110010111011000011001011010011011011100011101011110011111100000100001000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001000010000011000100000101000110000111001000001001001010001011001100001101001110001111010000",
    'SUMMARY_FILE':"/data_data/reinforcement_learning/results/summary_file.tsv",
    'HISTORY_FILE':None,
    'TAKE_KING_REWARD':10,
    'MORE_POINTS_REWARD':1,
    'EQUAL_POINTS_REWARD':0,
    'STEP_REWARD':-1,
    'WINS_DRAWS_LOSSES': [0,0,0],
    'in_channels_1':1,
    'in_channels_2':64,
    'in_channels_l':128,
    'out_channels_1':64,
    'out_channels_2':128,
    'out_channels_l':16,
    'kernel_1_size':(6,1),
    'kernel_2_size':(1,1),
    'kernel_l_size':(1,1),
    'stride_1_size':(6,1),
    'stride_2_size':(6,1),
    'stride_l_size':(1,1),
    'reshape_size':(8,48),
    'reshape_buffer':(1,384)
}

Overwriting args.py


In [8]:
%%writefile sparse_action_dict.py

sparse_action_dict = {'1,1,0':'010010011001','2,1,-2':'010011010111','2,1,2':'010011011011','2,2,-1':'010011100000','2,2,1':'010011100010','3,1,-1':'010100011000','3,1,1':'010100011010','4,1,-1':'010101011000','4,1,0':'010101011001','4,1,1':'010101011010','5,1,-1':'010110011000','5,1,0':'010110011001','5,1,1':'010110011010','6,1,-1':'010111011000','6,1,1':'010111011010','7,1,-2':'011000010111','7,1,2':'011000011011','7,2,-1':'011000100000','7,2,1':'011000100010','8,1,0':'011001011001','9,1,0':'011010011001','9,1,-1':'011010011000','9,1,1':'011010011010','10,1,0':'011011011001','10,1,-1':'011011011000','10,1,1':'011011011010','11,1,0':'011100011001','11,1,-1':'011100011000','11,1,1':'011100011010','12,1,0':'011101011001','12,1,-1':'011101011000','12,1,1':'011101011010','13,1,0':'011110011001','13,1,-1':'011110011000','13,1,1':'011110011010','14,1,0':'011111011001','14,1,-1':'011111011000','14,1,1':'011111011010','15,1,0':'100000011001','15,1,-1':'100000011000','15,1,1':'100000011010','16,1,0':'100001011001','16,1,-1':'100001011000','16,1,1':'100001011010','-16,-1,0':'000001001001','-16,-1,-1':'000001001000','-16,-1,1':'000001001010','-15,-1,0':'000010001001','-15,-1,-1':'000010001000','-15,-1,1':'000010001010','-14,-1,0':'000011001001','-14,-1,-1':'000011001000','-14,-1,1':'000011001010','-13,-1,0':'000100001001','-13,-1,-1':'000100001000','-13,-1,1':'000100001010','-12,-1,0':'000101001001','-12,-1,-1':'000101001000','-12,-1,1':'000101001010','-11,-1,0':'000110001001','-11,-1,-1':'000110001000','-11,-1,1':'000110001010','-10,-1,0':'000111001001','-10,-1,-1':'000111001000','-10,-1,1':'000111001010','-9,-1,0':'001000001001','-9,-1,-1':'001000001000','-9,-1,1':'001000001010','-8,-1,0':'001001001001','-7,-1,-2':'001010000111','-7,-1,2':'001010001011','-7,-2,-1':'001010000000','-7,-2,1':'001010000010','-6,-1,-1':'001011001000','-6,-1,1':'001011001010','-5,-1,-1':'001100001000','-5,-1,0':'001100001001','-5,-1,1':'001100001010','-4,-1,-1':'001101001000','-4,-1,0':'001101001001','-4,-1,1':'001101001010','-3,-1,-1':'001110001000','-3,-1,1':'001110001010','-2,-1,-2':'001111000111','-2,-1,2':'001111001011','-2,-2,-1':'001111000000','-2,-2,1':'001111000010','-1,-1,0':'010000001001'}

Overwriting sparse_action_dict.py


In [9]:
Pieces = {
    'Pawn':("w_P","b_P"),
    'Knight':("w_K","b_K"),
    'King':("w__K","b__K"),
    'Queen':("w__Q","b__Q"),
    'Bishop':("w_B","b_B"),
    'Rook':("w_R","b_R"),
}
    

In [10]:
  def getBin(num):
        if int(num) != 0:
            return "{0:{fill}6b}".format(int(num)+17, fill='0')
        else:
            return "{0:{fill}6b}".format(0, fill='0')

In [11]:
#sparse_action_dict

In [12]:
best_moves   = 0
policy_moves = 0
random_moves = 0

In [13]:
def getFunc(piece):
    def piece_func():
        return piece
    return piece_func()

def Bishop():
    return getFunc("Bishop")

def Pawn():
    return getFunc("Pawn")

def King():
    return getFunc("King")

def Queen():
    return getFunc("Queen")

def Knight():
    return getFunc("Knight")

def Rook():
    return getFunc("Rook")


In [14]:
Pawn()

'Pawn'

In [15]:

class Incorrect_Input_error(Exception):
  """Generic input error handler: raised in the case that any of the user inputed data is incorrect"""
  pass



In [16]:
temp0  = None
temp1  = None

In [17]:
from args import args
from sparse_action_dict import sparse_action_dict
from numeric_names import numeric_names

import torch
import torchvision


#from tensorboardX import SummaryWriter
import sys

sys.path.append('./anaconda3/lib/python3.7/site-packages/torchvision')
from tensorboardX import SummaryWriter
writer = SummaryWriter(comment="MADDPG Chessy")

id_names = {value:key for key,value in numeric_names.items()}
names    = {value: key for key in Pieces for value in Pieces[key]}


stats_rewards_list = []

def run_trial(user_input, env, mCritic, userInput, prev_wins_draws_losses):
    global temp0
    global temp1
    global cycle
    global scores
    global total_reward_agent
    global total_reward_env
    
    for j in range(user_input.num_sides):
        team_name = "team_" + str(j)
        team_name = Team(user_input.game, user_input.teams[j]["team_name"], j,user_input.teams[j]["skill"] / 10, user_input.teams[j]["strategy"])

        env.insert_team(team_name)
            
    env.not_deadlocked  = True
    env.states.append(args['initial_state'])
    time_step    = 0
        
    total_reward_agent = 0
    
    total_reward_env   = 0
        
    while env.not_deadlocked:
         
        curr_board_ids = {value:key for key,value in env.board.items()}
        state   = env.state 
        state   = np.array(list(map(int, state))).astype(np.float32)
        
        #state_prior_tensor = torch.IntTensor(state_prior_bin).to(device)
        if time_step % 2 == 0:
            action_ = agent_p0.act(state)
        else:
            action_ = agent_p1.act(state)

        #Determine feasible moves for each team:
        feasible_moves     = []
        feasible_moves_0   = env.get_feasible_moves(env.team[0])
        feasible_moves_1   = env.get_feasible_moves(env.team[1])
        [feasible_moves.append(x) for x in feasible_moves_0]
        [feasible_moves.append(x) for x in feasible_moves_1]
        
        f_moves   = set([int(x) for x in [action_id_dict[str((args['numeric_names'][env.board[curr_pos]],move)).replace("(","").replace(")","").replace(" ","")] for name, move, curr_pos, new_pos in feasible_moves]])
        
        f_moves_sparse = [id_action_dict[f_move] for f_move in f_moves]
        
        #f_moves  = set(np.sort(f_moves, axis=-1, kind='quicksort', order=None))
        
        action_   = [v if k in f_moves else 0 for k,v in enumerate(action_)]

        mid_point        = int((args['action_size'])/2)

        action_idx_0     = np.argmax(action_[:mid_point])
        action_idx_1     = np.argmax(action_[mid_point:]) + mid_point
        
        #action_idx_1     = np.argmax(action_[-mid_point:]) 

        action_sparse_0  = id_action_dict[action_idx_0]
        action_sparse_1  = id_action_dict[action_idx_1]
        #action_verbose_0 = (curr_board_ids[action_sparse_0[0]],action_sparse_0[1:])
        
        action_one_hot   = np.zeros(args['action_size'])
        
        action_one_hot[action_idx_0]   = 1
        action_one_hot[action_idx_1]   = 1
        
        env.best_moves_sparse          = [action_sparse_0,action_sparse_1]
        
        
        #############################################
        #  SEPARATE OUT INTO A STANDALONE FUNCTION  #
        #############################################
        
        
        def get_verbose_action(sp_action,i):

            act      = sp_action.split(",")
            
            name     = id_names[int(act[0])]

            curr_pos = curr_board_ids[name]
                
            next_pos = zip(curr_pos,action)

            move     = tuple([int(x) for x in act[1:]])

            next_pos = tuple(sum(tuples) for tuples in zip(curr_pos,move))
            
            player   = env.team[i].players[name]
                
            moves    = [player,move, curr_pos,next_pos]
            
            return moves

        
        #############################################
        # 
        #############################################
        
        try:
            env.best_moves_verbose         = [get_verbose_action(action_sparse_0,0),get_verbose_action(action_sparse_1,1)]
        except:
            pass
        
        next_state, reward, done, info = env.step(cycle,user_input,args)
        
        next_state                     = np.array(list(map(int, env.state))).astype(np.float32)
        
        total_reward_agent             += reward[0]
        total_reward_env               += reward[1]
        
        if userInput.teams[0]["skill"] > 1:
            agent_p0.step(state, action_one_hot, int(reward[0]), next_state, done, mCritic)
        
        if userInput.teams[1]["skill"] > 1:
            agent_p1.step(state, action_one_hot, int(reward[1]), next_state, done, mCritic)
        
        if cycle > 10:
            num_iter = cycle
            writer.add_scalar('Actor_loss', agent_p0.actorLoss, num_iter)
            writer.add_scalar('Critic_loss', agent_p0.mCriticLoss, num_iter)
            writer.add_scalar('Total_reward_agent', total_reward_agent, num_iter)
            writer.add_scalar('Total_reward_env', total_reward_env , num_iter)
            writer.add_scalar('Agent % Wins/(Wins + Losses + Draws)', args['WINS_DRAWS_LOSSES'][0]/sum(args['WINS_DRAWS_LOSSES']) , num_iter)
            writer.add_scalar('Agent % Losses/(Wins + Losses + Draws)', args['WINS_DRAWS_LOSSES'][2]/sum(args['WINS_DRAWS_LOSSES']) , num_iter)
            writer.add_scalar('Agent % Draws/(Wins + Losses + Draws)', args['WINS_DRAWS_LOSSES'][1]/sum(args['WINS_DRAWS_LOSSES']) , num_iter)
            writer.add_scalar('Wins net Losses)', (args['WINS_DRAWS_LOSSES'][0] - prev_wins_draws_losses[0]) - (args['WINS_DRAWS_LOSSES'][2] - prev_wins_draws_losses[2]), num_iter)
            writer.add_scalar('Episode Length', stats_rewards_list[-1][3], num_iter)
            writer.add_scalar('Avg Episode Length', np.mean(stats_rewards_list[-10:], axis=0)[3], num_iter)

        time_step                      += 1

    scores.append((cycle, info[0],info[1]))
    
    stats_rewards_list.append((cycle, total_reward_agent, total_reward_env, time_step))
    
    if cycle % args['update_every'] == 0:
            print("Agent scores: {}".format(np.mean(agent_scores[-args['window_size']:], axis=0)))
            
    if time_step > args['window_size'] and cycle % args['update_every'] == 0:
        
        torch.save(agent_p0.actor_network.state_dict(), 'checkpoint_p0_twoSided.pth')
        torch.save(agent_p1.actor_network.state_dict(), 'checkpoint_p1_twoSided.pth')
        torch.save(mCritic.network.state_dict(), 'checkpoint_mCritic_twoSided.pth')
        
        agent_win_margin = np.mean(stats_rewards_list[-args['window_size']:],  axis=0)[1] - np.mean(stats_rewards_list[-args['window_size']:],  axis=0)[2]
        print('Episode: {}'.format(cycle),
        'Timestep: {}'.format(cycle),
        'Total reward Agent: {}'.format(np.mean(stats_rewards_list[-args['window_size']:],  axis=0)[1]),
        'Total reward Env: {}'.format(np.mean(stats_rewards_list[-args['window_size']:],  axis=0)[2]),
        'Agent Win Margin: {}'.format(agent_win_margin),
        'Agent Wins: {}'.format(args['WINS_DRAWS_LOSSES'][0]),
        'Agent Draws: {}'.format(args['WINS_DRAWS_LOSSES'][1]),
        'Agent Losses: {}'.format(args['WINS_DRAWS_LOSSES'][2]),
        'Episode length: {}'.format(np.mean(stats_rewards_list[-args['window_size']:],axis=0)[3]),
        'mCriticLoss: {}'.format(agent_p0.mCriticLoss),
        'actorLoss: {}'.format(agent_p0.actorLoss))
        

In [18]:
agent_scores = []

In [19]:
scores = []

In [20]:
env = None

In [21]:


def run_trials(user_input):
    """ Runs the num_trials """
    global cycle
    global HISTORY_FILE
    global SUMMARY_FILE
    global env
    global agent_scores
    
    print("==============================RUN_TRIALS==================================")
    
    t = time.localtime()
    timestamp = time.strftime('%b_%d_%Y_%H%M', t)
    num_trials = user_input.num_trials
    num_sides = user_input.num_sides
    
    #SUMMARY_FILE = ("/data_data/reinforcement_learning/results/summary_file.tsv")

    #HISTORY_FILE = ("/data_data/reinforcement_learning/results/history_file_" + str(num_trials) + "_trials_" + str(num_sides) + "_sides_" + str(timestamp))
    
    """ Clear output file """
    
    args['HISTORY_FILE'] = "/data_data/reinforcement_learning/results/history_file_" + str(num_trials) + "_trials_" + str(num_sides) + "_sides_" + str(timestamp)
    
    with open(args['HISTORY_FILE'], "w") as history_file:
        history_file.write("\n")
        
        
    for i in range(num_trials):
        
        sides = [x for x in range(num_sides)]
        
        env = Game(user_input.game, 8, sides, user_input.display_board_positions)

        run_trial(user_input,env, mCritic, user_input, args['WINS_DRAWS_LOSSES'])

        cycle += 1
        
        agent_scores.append(env.last_reward)
  
        
    
   


In [22]:
from agent import Agent
from model import ActorNetwork, CriticNetwork, MCritic
from replaybuffer import ReplayBuffer
from args import args

mCritic       =  MCritic(args['state_size'],args['action_size'])
agent_p0      =  Agent(args['state_size'],args['action_size'], 0)
agent_p1      =  Agent(args['state_size'],args['action_size'], 1)

In [23]:
print(mCritic.network)
print(agent_p0.actor_network)

CriticNetwork(
  (fc1): Linear(in_features=472, out_features=10240, bias=True)
  (fc2): Linear(in_features=10240, out_features=4096, bias=True)
  (fc3): Linear(in_features=4096, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=1, bias=True)
)
ActorNetwork(
  (fc1): Linear(in_features=384, out_features=10240, bias=True)
  (fc2): Linear(in_features=10240, out_features=4096, bias=True)
  (fc3): Linear(in_features=4096, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=88, bias=True)
)


In [ ]:

from agent import Agent
from model import ActorNetwork, CriticNetwork, MCritic
from replaybuffer import ReplayBuffer
from plotutils import plot_results

from games  import games
from games import games
from game   import Game
from player import Player_Template
from team   import Team
from userinput import userInput

if __name__ == "__main__":
    global cycle
    global HISTORY_FILE
    user_input = userInput()
    run_trials(user_input)
    
    

#main()



select a game: ['chessy' or 'checkers']: chessy
select number of teams:  [0, 1 or 2] 2
Choose a name for this team [e.g. 'blue_team' or 'green_team']: Agent
Choose a color for team_2 [e.g. 'blue': green
Please enter team's skill_level [1 = Novice, 10 = expert]: 10
Please enter team's strategy [0 = 'cooperative', 1 = 'competitive']: 1
Choose a name for this team [e.g. 'blue_team' or 'green_team']: Environment
Choose a color for team_2 [e.g. 'blue': red
Please enter team's skill_level [1 = Novice, 10 = expert]: 10
Please enter team's strategy [0 = 'cooperative', 1 = 'competitive']: 1
How many trials would you like to run? [1 - 1,000,000] 100000
Do you want to see the board positions in realtime? [ 'Yes' or 'No' ]no
==============================RUN_TRIALS==================================


/home/ubuntu/chessy/model.py:48: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(self.fc4(x))
/home/ubuntu/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/ubuntu/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Agent scores: nan
Agent scores: [-1.07  1.07]
Agent scores: [-1.23  1.23]
Agent scores: [-2.7  2.7]
Agent scores: [-0.72  0.72]
Episode: 400 Timestep: 400 Total reward Agent: -78.78 Total reward Env: -77.34 Agent Win Margin: -1.4399999999999977 Agent Wins: 368 Agent Draws: 0 Agent Losses: 434 Episode length: 79.06 mCriticLoss: 0.8797740936279297 actorLoss: 0.9971776008605957
Agent scores: [-3.45  3.45]
Episode: 500 Timestep: 500 Total reward Agent: -69.59 Total reward Env: -62.87 Agent Win Margin: -6.720000000000006 Agent Wins: 438 Agent Draws: 0 Agent Losses: 564 Episode length: 67.23 mCriticLoss: 1.468734860420227 actorLoss: 1.0012733936309814
Agent scores: [-1.39  1.39]
Agent scores: [-3.03  3.03]
Agent scores: [-3.97  3.97]
Agent scores: [-1.78  1.78]
Agent scores: [-3.56  3.56]
Agent scores: [-2.35  2.35]
Agent scores: [-2.78  2.78]
Agent scores: [-3.45  3.45]
Episode: 1300 Timestep: 1300 Total reward Agent: -64.18 Total reward Env: -57.1 Agent Win Margin: -7.080000000000005 Agent

Agent scores: [-2.79  2.79]
Episode: 9000 Timestep: 9000 Total reward Agent: -71.94 Total reward Env: -66.32 Agent Win Margin: -5.6200000000000045 Agent Wins: 7598 Agent Draws: 34 Agent Losses: 10370 Episode length: 70.13 mCriticLoss: 0.3906659483909607 actorLoss: 1.0035724639892578
Agent scores: [-2.45  2.45]
Episode: 9100 Timestep: 9100 Total reward Agent: -70.67 Total reward Env: -65.77 Agent Win Margin: -4.900000000000006 Agent Wins: 7678 Agent Draws: 34 Agent Losses: 10490 Episode length: 69.22 mCriticLoss: 0.981993556022644 actorLoss: 0.9987435340881348
Agent scores: [-1.01  1.01]
Agent scores: [-3.22  3.22]
Agent scores: [-1.61  1.61]
Agent scores: [-2.18  2.18]
Agent scores: [-2.71  2.71]
Episode: 9600 Timestep: 9600 Total reward Agent: -64.33 Total reward Env: -58.73 Agent Win Margin: -5.600000000000001 Agent Wins: 8112 Agent Draws: 40 Agent Losses: 11050 Episode length: 62.53 mCriticLoss: 1.4668018817901611 actorLoss: 0.9921092987060547
Agent scores: [-2.92  2.92]
Episode: 97

Agent scores: [-1.44  1.44]
Agent scores: [-0.16  0.16]
Agent scores: [-3.17  3.17]
Agent scores: [-1.72  1.72]
Agent scores: [-2.52  2.52]
Agent scores: [-1.37  1.37]
Agent scores: [-2.43  2.43]
Agent scores: [-2.5  2.5]
Agent scores: [-2.93  2.93]
Episode: 18100 Timestep: 18100 Total reward Agent: -73.98 Total reward Env: -68.34 Agent Win Margin: -5.640000000000001 Agent Wins: 14992 Agent Draws: 78 Agent Losses: 21132 Episode length: 72.16 mCriticLoss: 1.8608219623565674 actorLoss: 0.9992356896400452
Agent scores: [-0.21  0.21]
Agent scores: [-1.24  1.24]
Episode: 18300 Timestep: 18300 Total reward Agent: -77.74 Total reward Env: -75.48 Agent Win Margin: -2.259999999999991 Agent Wins: 15200 Agent Draws: 78 Agent Losses: 21324 Episode length: 77.61 mCriticLoss: 1.8583972454071045 actorLoss: 1.0005505084991455
Agent scores: [-1.67  1.67]
Episode: 18400 Timestep: 18400 Total reward Agent: -66.83 Total reward Env: -63.49 Agent Win Margin: -3.3399999999999963 Agent Wins: 15286 Agent Draws

In [ ]:
agent_p0.actor_network.actor_loss

In [ ]:
score_board = []
for i in [0,1]:
    score_board.append([(x[0],x[i+1]) for x in scores])
#score_board

In [ ]:
  team_scores = []
  min_score = 99999999
  max_score = 0

  for j in range(user_input.num_sides):
    team_scores.append([team_points for cycle, team_points in score_board[j]])
    plot.hist(
      team_scores[j],
      bins=100,
      label=user_input.teams[j]["team_name"],
      color=user_input.teams[j]["color"])

    min_j_score = min(team_scores[j])
    max_j_score = max(team_scores[j])

In [ ]:
wins   = args['WINS_DRAWS_LOSSES'][0]
draws  = args['WINS_DRAWS_LOSSES'][1]
losses = args['WINS_DRAWS_LOSSES'][2]

In [ ]:
 args['WINS_DRAWS_LOSSES']

In [ ]:
plot.hist(wins)


In [ ]:
torch.save(agent_p0.actor_network.state_dict(), 'checkpoint_p0_oneSided.pth')
torch.save(agent_p1.actor_network.state_dict(), 'checkpoint_p1_oneSided.pth')
torch.save(mCritic.network.state_dict(), 'checkpoint_mCritic_oneSided.pth')